In [4]:
from docplex.mp.model import Model
import time

In [5]:
nodos=7

b=[]
aux=[]
for i in range(nodos):
     aux.append(0) 
for i in range(nodos):
     b.append(aux[:])  
aux=1        
for i in range(nodos):
    for j in range(nodos):
        if(i != j):
            b[i][j]=50
            aux=aux+1           
c=[]
aux=[]
for i in range(nodos):
     aux.append(0) 
for i in range(nodos):
     c.append(aux[:])  
for i in range(nodos):
    for j in range(nodos):
        if(i != j):
            c[i][j]=abs(i-j)*10-abs(i-j)+1 
            
B2={(o,d,i):0 for o in range(nodos) for d in range(nodos) for i in  range(nodos)}
for i in range(nodos):
    for j in range(nodos):
        B2[i,j,i]=b[i][j]
        B2[i,j,j]=-b[i][j]

rutas=[(a,b,c,d,e,f,g) for a in range(2)for b in range(2)for c in range(2)for d in range(2)for e in range(2)for f in range(2)for g in range(2)]
#rutas=[]
#rutas.append((0,1,0,1)) 
#rutas.append((0,1,1,0)) 
#rutas.append((1,0,0,1)) 
#rutas.append((1,0,1,0)) 
#rutas.append((1,0,1,1)) 
#rutas.append((1,1,0,1))

tamRutas=len(rutas)

arcos1=[(k,i,j)for k in range(tamRutas) for i in range( nodos) for j in range(nodos) ]
rutasmodificadas=[]

rutaaux=[]

for i in range(tamRutas):
    rutaaux.clear() 
    for j in range(nodos):
        if rutas[i][j] == 1:
            rutaaux.append(j)          
    rutasmodificadas.append(rutaaux[:])  


r={}
for i in arcos1:
    r[i]=0
for i in range(tamRutas):
    for j in range(len(rutasmodificadas[i])-1):
        r[(i,rutasmodificadas[i][j],rutasmodificadas[i][j+1])]=1
        r[(i,rutasmodificadas[i][j+1],rutasmodificadas[i][j])]=1
        
print(b)

[[0, 50, 50, 50, 50, 50, 50], [50, 0, 50, 50, 50, 50, 50], [50, 50, 0, 50, 50, 50, 50], [50, 50, 50, 0, 50, 50, 50], [50, 50, 50, 50, 0, 50, 50], [50, 50, 50, 50, 50, 0, 50], [50, 50, 50, 50, 50, 50, 0]]


In [6]:

from docplex.mp.model import Model
mdl= Model("modelo")
arcos=[(i,j,o,d) for i in range(nodos)for j in range(nodos)for o in range(nodos)for d in range(nodos)]
x=mdl.continuous_var_dict(arcos,name="x")
arcos2=[(i) for i in range (tamRutas)]
y=mdl.binary_var_dict(arcos2,name="y")
f=mdl.continuous_var_dict(arcos2,name="f")


u= 20
m= 60
Q= 50
alpha=100




mdl.minimize(mdl.sum(c[i][j]*x[(i,j,o,d)] for i in range(nodos)for j in range(nodos)for o in range(nodos)for d in range(nodos))+mdl.sum(alpha*(f[(i)]+y[i])for i in range(tamRutas)))
 

    
for i in  range(nodos):
    for j in range(nodos):
        mdl.add_constraint(mdl.sum(x[(i,j,o,d)] for o in range(nodos)for d in range(nodos))<= mdl.sum(r[(k,i,j)]*f[(k)]*u for k in range(tamRutas)))

for i in range(nodos):  
    for o in range(nodos):
        for d in range(nodos):       
            mdl.add_constraint((mdl.sum(x[(i,j,o,d) ] for j in range(nodos) )-mdl.sum(x[(j,i,o,d) ] for j in range(nodos) ) )==B2[(o,d,i)])


for i in  range(tamRutas):
    mdl.add_constraint(f[(i)]<=y[(i)]*10)
    
mdl.add_constraint(mdl.sum(y[(k)]for k in range(tamRutas))<=m)  
mdl.add_constraint(mdl.sum(f[(k)]for k in range(tamRutas))<=Q)   
inicio = time.time()
solucion=mdl.solve(log_output=True)
fin = time.time()
print(fin-inicio)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 15 rows and 359 columns.
Reduced MIP has 507 rows, 2298 columns, and 7296 nonzeros.
Reduced MIP has 120 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.89 ticks)
Found incumbent of value 82820.000000 after 0.01 sec. (9.23 ticks)
Probing time = 0.00 sec. (3.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 42 rows and 252 columns.
Reduced MIP has 465 rows, 2046 columns, and 6540 nonzeros.
Reduced MIP has 120 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.08 ticks)
Probing time = 0.00 sec. (2.49 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 24 threads.
Root relaxation solution time = 0.01 sec. (3.69 ticks)

        Nodes                                        

In [7]:
solucion=mdl.solve(log_output=True)  
archivo = open("aproximacion-Experimento", "w") 
archivo.write(str(solucion)) 
archivo.close()
#m=10            

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.04 ticks)
Parallel b&c, 24 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.04 ticks)


In [9]:

for i in range(tamRutas):
    if(f[ (i) ].solution_value>0 and y[ (i) ].solution_value>0):
        print("rutas.append(",end="")
        print(rutas[i],end="")
        print(")")

rutas.append((1, 0, 0, 0, 1, 0, 1))
rutas.append((1, 0, 0, 1, 0, 1, 0))
rutas.append((1, 0, 1, 0, 0, 1, 1))
rutas.append((1, 0, 1, 1, 0, 0, 1))
rutas.append((1, 1, 0, 0, 0, 0, 1))
rutas.append((1, 1, 0, 0, 0, 1, 0))
rutas.append((1, 1, 0, 0, 1, 0, 0))
rutas.append((1, 1, 0, 1, 1, 0, 0))
rutas.append((1, 1, 1, 0, 0, 0, 1))
rutas.append((1, 1, 1, 0, 1, 1, 1))


(1, 0, 0, 1) 10.0
(1, 0, 1, 1) 10.0
(1, 1, 0, 1) 10.0
(1, 1, 1, 0) 10.0


Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Retaining values of one MIP start for possible repair.
Tried aggregator 1 time.
MIP Presolve eliminated 9 rows and 74 columns.
Reduced MIP has 88 rows, 214 columns, and 643 nonzeros.
Reduced MIP has 11 binaries, 203 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.29 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Cover probing fixed 0 vars, tightened 96 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 11 rows and 11 columns.
Reduced MIP has 77 rows, 203 columns, and 621 nonzeros.
Reduced MIP has 11 binaries, 192 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.43 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 24 threads.
Root relaxation solution time = 0.00 sec. (0.19 ticks)

        Nodes          

In [12]:
print(rutasmodificadas[6])
print(rutasmodificadas[9])
print(rutasmodificadas[11])
print(rutasmodificadas[13])



[1, 2]
[0, 3]
[0, 2, 3]
[0, 1, 3]


In [2]:
print(c)

[[0, 10, 19, 28, 37, 46, 55, 64], [10, 0, 10, 19, 28, 37, 46, 55], [19, 10, 0, 10, 19, 28, 37, 46], [28, 19, 10, 0, 10, 19, 28, 37], [37, 28, 19, 10, 0, 10, 19, 28], [46, 37, 28, 19, 10, 0, 10, 19], [55, 46, 37, 28, 19, 10, 0, 10], [64, 55, 46, 37, 28, 19, 10, 0]]


In [43]:
print(b)


[[0, 10, 20, 30], [40, 0, 50, 60], [70, 80, 0, 90], [100, 110, 120, 0]]
